In [37]:
import pandas as pd
from assembly_tools import load_experimental_grid, format_table, extract_raw_performance
from IPython.display import display, HTML


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from hydra import initialize, compose
from omegaconf import OmegaConf

### Assembly of experimental results 2


In [12]:
# load default params. They can be changed however you want.
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name='extract.yaml')
    print(cfg)
general_params =  ["resolution_value", "ds_name", "method_hps.max_lag"]
hp_stack = []
performance_stack = []
scoring = 'Max individual F1'
cfg.ds_order = ["random_5"]

{'metrics': ['Max Acc thresh', 'Max Acc', 'Max individual Acc thresh', 'Max individual Acc', 'Null Acc', 'Max F1 thresh', 'Max F1', 'Max individual F1 thresh', 'Max individual F1', 'Null F1', 'AUROC', 'Individual AUROC', 'Null AUROC', 'runtime'], 'hp_list': ['data_preprocess.resolution', 'data_preprocess.normalize', 'label_path', 'method.max_lag', 'method.var_absolute_values', 'method.filter_mode', 'method.architecture'], 'rename': {'resolution_value': 'data_preprocess.resolution', 'method_hps.var_absolute_values': 'method.var_absolute_values', 'method_hps.filter_mode': 'method.filter_mode', 'method_hps.max_lag': 'method.max_lag', 'method_hps.cp_architecture': 'method.architecture', 'ds_name': 'label_path', 'normalize': 'data_preprocess.normalize', 'window_data_month_value': 'data_preprocess.subset_month', 'window_data_year_value': 'data_preprocess.subset_year'}, 'ds_order': ['close_3', 'close_5', 'root_cause_3', 'root_cause_5', '1_random_3', '1_random_5', 'confounder_3', 'confounder_5

In [13]:
for name in ["reverse", "cross", "combo", "var_", "varlingam","pcmci","dyno","cp"]:
    data = pd.read_csv("grid_export_exp2/" + name + ".csv", index_col=0)
    res = ["data_path",  "data_preprocess.subset_year"]
    formatted = format_table(data,cfg, scoring= scoring, restriction=res)
    lines = extract_raw_performance(formatted,cfg,scoring,name="", restriction=res)
    lines.columns = [name]
    hp_stack.append(formatted)
    performance_stack.append(lines)

Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')


In [ ]:
data = load_experimental_grid(mypath=cfg.data_path + cfg.exp, method_name=cdmi).T

,2025-01-10 15:33:40.8086,2025-01-10 15:56:59.5209,2025-01-10 15:33:15.7776,2025-01-10 15:57:15.9363,2025-01-10 15:35:05.0635,2025-01-10 16:34:02.9783,2025-01-10 15:57:46.8058,2025-01-10 15:36:28.3311,2025-01-10 15:38:18.3771,2025-01-10 15:58:45.5143,...,2025-01-10 15:55:31.9040,2025-01-10 15:39:43.8402,2025-01-10 15:59:11.7795,2025-01-10 15:45:18.2360,2025-01-10 15:46:40.2374,2025-01-10 16:00:08.3107,2025-01-10 15:43:52.2987,2025-01-10 15:42:29.0037,2025-01-10 15:46:37.0810,2025-01-10 15:45:15.5916
Max Acc thresh,0.729985,0.020962,0.578205,0.024432,0.279939,0.016472,0.016167,0.399911,0.339914,0.21861,...,0.99,0.119992,0.039963,1.0,1.0,0.999784,1.000001,1.000001,1.000001,1.000001
Max Acc,0.801192,0.800281,0.808649,0.800187,0.805984,0.801706,0.801613,0.807901,0.806685,0.799977,...,0.801075,0.802665,0.804208,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813
Max individual Acc thresh,0.040654,0.000695,0.152619,0.00071,0.107314,0.004983,0.005238,0.133028,0.111938,0.002521,...,0.466994,0.030642,0.040081,0.710101,0.655612,0.046434,0.08949,0.08033,0.088316,0.119423
Max individual Acc,0.817204,0.813838,0.831206,0.815077,0.833941,0.8295,0.830248,0.83532,0.83504,0.81676,...,0.827466,0.828121,0.83878,0.815778,0.817041,0.837144,0.828752,0.828284,0.8313,0.832352
Null Acc,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,...,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813,0.799813
Max F1 thresh,0.999637,0.0,0.996905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000041,0.0,0.908856,1.0,1.0,0.961048,1.0,1.0,0.999979,0.999819
Max F1,0.333626,0.333697,0.334008,0.333738,0.3336,0.333645,0.333684,0.333593,0.335239,0.336748,...,0.38812,0.370431,0.428008,0.438902,0.441733,0.460419,0.408555,0.39823,0.44074,0.403113
Max individual F1 thresh,0.054693,0.000301,0.099049,0.000437,0.113488,0.00571,0.005765,0.097565,0.103565,0.002022,...,0.434008,0.029038,0.009048,0.730159,0.708791,0.065125,0.038813,0.044287,0.18808,0.184766
Max individual F1,0.444072,0.447898,0.449708,0.454013,0.455917,0.457568,0.46151,0.461737,0.464725,0.474461,...,0.530792,0.533123,0.535662,0.54249,0.547624,0.548164,0.554426,0.558456,0.593356,0.60005
Null F1,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,...,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593,0.333593


data

In [34]:
out = pd.concat(performance_stack,axis=1).T
out = out[[out.columns[1], out.columns[0], out.columns[2]]]
out.columns = ["Full TS", "No Rain", "Flood"]
out.index = ["RP", "CC", "RPCC", "VAR", "Varlingam", "PCMCI", "Dynotears", "CP"]
out = out.astype(float).round(3)

In [38]:
style = out.style.highlight_max(axis=1,  props='color:black;font-weight:bold')
style = style.format("{:.2f}")
latex_str = style.to_latex(convert_css=True)
display(HTML(style.to_html()))
print(latex_str)

,Full TS,No Rain,Flood
RP,0.64,0.64,0.57
CC,0.61,0.51,0.59
RPCC,0.62,0.62,0.60
VAR,0.68,0.70,0.68
Varlingam,0.61,0.59,0.62
PCMCI,0.50,0.58,0.53
Dynotears,0.69,0.49,0.71
CP,0.55,0.53,0.60


\begin{tabular}{lrrr}
 & Full TS & No Rain & Flood \\
RP & \color{black} \bfseries 0.64 & \color{black} \bfseries 0.64 & 0.57 \\
CC & \color{black} \bfseries 0.61 & 0.51 & 0.59 \\
RPCC & 0.62 & \color{black} \bfseries 0.62 & 0.60 \\
VAR & 0.68 & \color{black} \bfseries 0.70 & 0.68 \\
Varlingam & 0.61 & 0.59 & \color{black} \bfseries 0.62 \\
PCMCI & 0.50 & \color{black} \bfseries 0.58 & 0.53 \\
Dynotears & 0.69 & 0.49 & \color{black} \bfseries 0.71 \\
CP & 0.55 & 0.53 & \color{black} \bfseries 0.60 \\
\end{tabular}



In [43]:
$\uparrow$

SyntaxError: invalid syntax (3520154587.py, line 1)

In [15]:
# TODO: Integrate single CDMI line.